# SIRE 
### stochastic impulse response emulator

import heaps of python

In [1]:
% matplotlib inline
%config InlineBackend.close_figures=False 
from __future__ import print_function
#from ipywidgets import interact, interactive, fixed, interact_manual
# import jtplot module in notebook
import os
import ipywidgets as ipyw
import matplotlib as mpl
import matplotlib.pyplot as plt
import ipyleaflet as ipyl
from datetime import datetime
import IPython
import warnings
warnings.filterwarnings("ignore")
import json
from json import dumps
import numpy as np
import pandas as pd
import pyemu

import hauraki_sire

no mprj


# Land Use Sector Map

In [2]:
# with open(os.path.join("sire","sire_sector.json"),"r") as f:
#     lu_data = json.load(f)
# labellu = ipyw.Label(layout=ipyw.Layout(width='100%'))
# def hover_handler(event=None, id=None, properties=None):
#     labellu.value = properties["secConcord"]

# geo = ipyl.GeoJSON(data=lu_data)
# geo.on_hover(hover_handler)
# mlu = ipyl.Map(center=[-37.65,175.65], zoom=10,layout=ipyw.Layout(height="10in",width="6in"))
# mlu.add_layer(geo);
# ipyw.VBox([labellu,mlu])

### A bunch of widget hackery

In [28]:
m_load = ipyl.Map(center=[-37.65,175.65], zoom=9,layout=ipyw.Layout(height="6in",width="3in"))
m_sfr = ipyl.Map(center=[-37.65,175.65], zoom=9,layout=ipyw.Layout(height="6in",width="3in"))
m_ucn = ipyl.Map(center=[-37.65,175.65], zoom=9,layout=ipyw.Layout(height="6in",width="3in"))


# with open(os.path.join("sire","sire_sfr.json"),"r") as f:
#     sfr_data = json.load(f)

# with open(os.path.join("sire","sire_grid.json"),"r") as f:
#     ucn_data = json.load(f)    
    
# with open(os.path.join("sire","sire_grid.json"),"r") as f:
#     load_data = json.load(f) 
# widget hackery
lu_mn,lu_mx = -10,10 #min and max landuse change
status_str = ipyw.Text()
risk_str = ipyw.Text()
dairy_str = ipyw.Text()
hort_str = ipyw.Text()
snb_str = ipyw.Text()
forestry_str = ipyw.Text()
other_str = ipyw.Text()
str_dict = {"dairy":dairy_str,
           "hort":hort_str,"snb":snb_str,
           "forestry":forestry_str,
           "other":other_str}
def set_values(**kwargs):
    for k,v in kwargs.items():
        str_dict[k].value = str(v)
def set_risk(risk):
    risk_str.value = str(risk)
w = ipyw.interactive(set_values,dairy=(lu_mn,lu_mx,1),
                hort=(lu_mn,lu_mx,1),snb=(lu_mn,lu_mx,1),
                forestry=(lu_mn,lu_mx,1),
                other=(lu_mn,lu_mx,1))

i = IPython.display.Audio("Dang.mp3",autoplay=True)

ax_lab_load = ipyw.Label(layout=ipyw.Layout(width='100%'))
ax_lab_load.value = "N loading change (delta kg/day)"
ax_lab_sw = ipyw.Label(layout=ipyw.Layout(width='100%'))
ax_lab_sw.value = "SW concentration change (delta mg/l)"
ax_lab_gw = ipyw.Label(layout=ipyw.Layout(width='100%'))
ax_lab_gw.value = "GW concentration change (delta mg/l)"

label_load = ipyw.Label(layout=ipyw.Layout(width='100%'))
def load_hover_handler(event=None, id=None, properties=None):
    label_load.value = "row:{0}, col:{1}, delta kg:{2:8.2G}".format(properties["row"],
                                                                           properties["column"],
                                                                           properties["response"])

label_sfr = ipyw.Label(layout=ipyw.Layout(width='100%'))
def sfr_hover_handler(event=None, id=None, properties=None):
    label_sfr.value = "reachid:{0}, response (mg/l):{1:8.2G}".format(properties["reachID"],
                                                                     properties["response"])

label_ucn = ipyw.Label(layout=ipyw.Layout(width='100%'))
def ucn_hover_handler(event=None, id=None, properties=None):
    label_ucn.value = "row:{0}, col:{1}, response (mg/l):{2:8.2G}".format(properties["row"],
                                                                           properties["column"],
                                                                          properties["response"])

plt.ioff()

fig_ucn = plt.figure(figsize=(4.5,4.5))
ax_ucn=plt.subplot(111)
out_ucn=ipyw.Output()    
def ucn_on_click(event=None, id=None, properties=None):
#     ax_unc.clear()
#     ax_unc.plot(np.random.randn(100),np.random.randn(100),'+')
#     ax_unc.set_xlim(-3,3)
#     ax_unc.set_ylim(-3,3)
#     ax_unc.set_yticklabels([])  
#     with out_unc:
#         IPython.display.clear_output(wait=True)
#         IPython.display.display(ax_unc.figure)
    #print(properties)
    ax = ax_ucn
    ax.clear()
    if properties is None:
        ax.set_xticklabels([])
        ax.set_yticklabels([])
        ax.text(0.5,0.5,"click on a cell\nto plot",ha="center",va="center",transform=ax.transAxes)
        
        with out_ucn:
            
            IPython.display.clear_output(wait=True)
            IPython.display.display(ax.figure)
        return
    
    #ax_sfr.plot(np.random.randn(100),np.random.randn(100),'+')
    #ax_sfr.set_xlim(-3,3)
    #ax_sfr.set_ylim(-3,3)
    resp,resp_org = properties["response"],properties["response_org"]
    risk,std = float(risk_str.value),properties["std"]
    c = properties["style"]["fillColor"]
    x,y = pyemu.plot.plot_utils.gaussian_distribution(resp_org,std)
    ax.fill_between(x,0,y,alpha=0.5,facecolor=c)
    ymn,ymx = ax.get_ylim()
    ax.plot([resp,resp],[ymn,ymx],"k--")
    ax.set_yticklabels([])
    ax.set_ylim(ymn,ymx)
    
    ax.set_title("row:{0}, col:{1}, std:{2:10.3f}".format(properties["row"],properties["column"],std))
    ax.set_xlabel("response ($\\Delta\\frac{mg}{l}$)")
    ax.grid()
    
    with out_ucn:
        IPython.display.clear_output(wait=True)
        IPython.display.display(ax.figure)


fig_sfr = plt.figure(figsize=(4.5,4.5))
ax_sfr=plt.subplot(111)
out_sfr=ipyw.Output()    
def sfr_on_click(event=None, id=None, properties=None):
    #print(properties)
    ax = ax_sfr
    ax.clear()
    if properties is None:
        ax.text(0.5,0.5,"click on a reach\nto plot",ha="center",va="center",transform=ax.transAxes)
        ax.set_xticklabels([])
        ax.set_yticklabels([])
        with out_sfr:
            
            IPython.display.clear_output(wait=True)
            IPython.display.display(ax.figure)
        return
    
    #ax_sfr.plot(np.random.randn(100),np.random.randn(100),'+')
    #ax_sfr.set_xlim(-3,3)
    #ax_sfr.set_ylim(-3,3)
    resp,resp_org = properties["response"],properties["response_org"]
    risk,std = float(risk_str.value),properties["std"]
    c = properties["style"]["fillColor"]
    x,y = pyemu.plot.plot_utils.gaussian_distribution(resp_org,std)
    ax.fill_between(x,0,y,alpha=0.5,facecolor=c)
    ymn,ymx = ax.get_ylim()
    ax.plot([resp,resp],[ymn,ymx],"k--")
    ax.set_yticklabels([])
    ax.set_ylim(ymn,ymx)
    ax.grid()
    ax.set_xlabel("response ($\\Delta\\frac{mg}{l}$)")
    ax.set_title("reach:{0}, std:{1:10.3f}".format(properties["reachID"],std))
    
    with out_sfr:
        IPython.display.clear_output(wait=True)
        IPython.display.display(ax.figure)

fig_cb = plt.figure(figsize=(4.5,4.5))
ax_cb_load=plt.subplot(311)
ax_cb_gw=plt.subplot(312)
ax_cb_sfr=plt.subplot(313)
out_cb=ipyw.Output()         
out_cb_gw=ipyw.Output()         

out_cb_sfr=ipyw.Output()         

gradient = np.linspace(0, 1, 256)
gradient = np.vstack((gradient, gradient))
        
def on_button_clicked(b):  
    status_str.value = ''
    tol = 1.0e-3
    risk = float(risk_str.value)
    #risk = 0.025
    status_str.value += "calculating..."
    start = datetime.now()
    lu_dict = {k:float(v.value) for k,v in str_dict.items()} 
    #status_str.value += "took: {0}".format((datetime.now()-s).total_seconds())
    status_str.value += "plotting..."
    #start = datetime.now()
    s = datetime.now()
    sfr_data,ucn_data,load_data, lim_dict = hauraki_sire.sire_lu_scenario_json(lu_dict,risk=risk)
    geo = ipyl.GeoJSON(data=load_data)
    geo.on_hover(load_hover_handler)
    m_load.add_layer(geo) 
    geo = ipyl.GeoJSON(data=sfr_data)
    geo.on_hover(sfr_hover_handler)
    geo.on_click(sfr_on_click)
    m_sfr.add_layer(geo) 
    geo = ipyl.GeoJSON(data=ucn_data)
    geo.on_hover(ucn_hover_handler)
    geo.on_click(ucn_on_click)
    m_ucn.add_layer(geo)
    sfr_on_click()
    ucn_on_click()

    ax_cb_load.clear()
    ax_cb_gw.clear()
    ax_cb_sfr.clear()
    mpl.colorbar.ColorbarBase(ax_cb_load,cmap="jet",orientation="horizontal")
#     with out_cb_load:
#         IPython.display.clear_output(wait=True)
#         IPython.display.display(ax_cb_load.figure)
#     with out_cb_gw:
#         IPython.display.clear_output(wait=True)
#         IPython.display.display(ax_gw_load.figure)
#     with out_cb_sfr:
#         IPython.display.clear_output(wait=True)
#         IPython.display.display(ax_cb_sfr.figure)
    with out_cb_load:
         IPython.display.clear_output(wait=True)
         IPython.display.display(fig_cb)
    status_str.value += "took:{0}".format((datetime.now() - start).total_seconds())
    

button = ipyw.Button(description="Get Some Science")
button.on_click(on_button_clicked)
print("ready!")
#i

ready!


# Let's do this!

In [29]:
print("risk = probability that the nitrate concentration change\n       is less than what is shown")
r = ipyw.interact(set_risk,risk=(0.01,0.99,0.01))
print("percent change in nitrate loading by land use sector")
display(w)
display(button)
display(status_str)
#ipyw.HBox([ipyw.VBox([labellu,mlu]),ipyw.VBox([label_sfr,m_sfr])])
ipyw.HBox([ipyw.VBox([ax_lab_load,label_load,m_load,out_cb_load,out_cb_gw,out_cb_sfr]),
           ipyw.VBox([ax_lab_gw,label_ucn,m_ucn,out_ucn]),
           ipyw.VBox([ax_lab_sw,label_sfr,m_sfr,out_sfr])])

risk = probability that the nitrate concentration change
       is less than what is shown


interactive(children=(FloatSlider(value=0.5, description='risk', max=0.99, min=0.01, step=0.01), Output()), _d…

percent change in nitrate loading by land use sector


interactive(children=(IntSlider(value=0, description='snb', max=10, min=-10), IntSlider(value=0, description='…

Button(description='Get Some Science', style=ButtonStyle())

Text(value='')